In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


/kaggle/input/sample_submission.csv
/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/description.md
/kaggle/input/sample_submission.csv.zip
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/description.md
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv')


In [4]:
train.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test.head()


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
train.isnull().any()


id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [7]:
test.isnull().any()


id              False
comment_text    False
dtype: bool

In [8]:
x_train = train['comment_text']
y_train = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
x_test = test['comment_text']


In [9]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)


2025-08-18 13:54:25.117671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755525265.130690       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755525265.134662       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 13:54:25.150484: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [10]:
x_tokenized_train = tokenizer.texts_to_sequences(x_train)
x_tokenized_test = tokenizer.texts_to_sequences(x_test)


NameError: name 'tokenizer' is not defined

In [11]:
lengths = [len(comment) for comment in x_tokenized_train]
print(f'The longest comment is {max(lengths)} words long.')
sns.distplot(lengths, kde=False)


NameError: name 'x_tokenized_train' is not defined

In [12]:
from keras.preprocessing.sequence import pad_sequences

max_length = 200
X_train = pad_sequences(x_tokenized_train, maxlen=max_length)
X_test = pad_sequences(x_tokenized_test, maxlen=max_length)


NameError: name 'x_tokenized_train' is not defined

In [13]:
len(tokenizer.word_index)


NameError: name 'tokenizer' is not defined

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GlobalAveragePooling1D, Dropout, Dense, LeakyReLU, Activation

num_features, embed_size = len(tokenizer.word_index), 128

models = []

models += [Sequential(), Sequential(), Sequential()]

models[0].add(Embedding(num_features + 1, embed_size, input_length=max_length))
models[0].add(LSTM(64, return_sequences=True))
models[0].add(GlobalAveragePooling1D())
models[0].add(Dropout(0.1))
models[0].add(Dense(48))
models[0].add(LeakyReLU())
models[0].add(Dropout(0.1))
models[0].add(Dense(6, activation='sigmoid'))
models[0].compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

models[1].add(Embedding(num_features + 1, embed_size, input_length=max_length))
models[1].add(LSTM(64, return_sequences=True))
models[1].add(GlobalAveragePooling1D())
models[1].add(Dropout(0.1))
models[1].add(Dense(48))
models[1].add(Activation('relu'))
models[1].add(Dropout(0.1))
models[1].add(Dense(6, activation='sigmoid'))
models[1].compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

models[2].add(Embedding(num_features + 1, embed_size, input_length=max_length))
models[2].add(LSTM(64, return_sequences=True))
models[2].add(GlobalAveragePooling1D())
models[2].add(Dropout(0.1))
models[2].add(Dense(32))
models[2].add(Activation('relu'))
models[2].add(Dropout(0.1))
models[2].add(Dense(16))
models[2].add(Activation('relu'))
models[2].add(Dropout(0.1))
models[2].add(Dense(6, activation='sigmoid'))
models[2].compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

models


NameError: name 'tokenizer' is not defined

In [15]:
batch_size = 4096
validation_split = 0.1
epochs = 1

histories = []

for model in models:
    print(model.summary())
    history = model.fit(X_train, y_train,
                        validation_split=validation_split,
                        batch_size=batch_size,
                        epochs=epochs)
    histories.append(history)


NameError: name 'models' is not defined

In [16]:
y_preds = []

for i, model in enumerate(models):
    print(f'Started predicting for model {i}')
    y_pred = model.predict(X_test, batch_size=4096)
    y_preds.append(y_pred)
    print(f'Predicted stuff for model {i}')


NameError: name 'models' is not defined

In [17]:
y = []
for i, model in enumerate(models):
    y_i = pd.DataFrame(data=y_preds[i],
                        columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
    y_i = pd.concat([test['id'], y_i], axis=1)
    y.append(y_i)
# y = pd.DataFrame(data=y_pred, columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
# y = pd.concat([test['id'], y], axis=1)
y


NameError: name 'models' is not defined

In [18]:
for i, y_i in enumerate(y):
    filename = f'submision_{i}.csv'
    y_i.to_csv(filename, index=False)
    print(f'Created file {filename}')
# y.to_csv('submission.csv', index=False)
